In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
from datetime import datetime
from tqdm.auto import tqdm
import pymysql

# 크롬 드라이버 자동 Install
ChromeDriverManager().install()


/Users/mac/Desktop/JoonHo/OZ_Backend_School_miniProject/Crwaling_book_info/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/mac/.wdm/drivers/chromedriver/mac64/124.0.6367.60/chromedriver-mac-x64/chromedriver'

In [2]:
browser = webdriver.Chrome()
path = './chromedriver' #크롬 드라이버 저장 경로
#Yes24 베스트 셀러를 1페이지에 120개씩 볼수 있도록 URL 설정
# browser.get('https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=120') #URL로 이동
# #페이지가 로딩 될때까지 10초간 대기, 로딩이 완료되면 다음 코드 즉시 실행
# browser.implicitly_wait(time_to_wait=1000) 



In [3]:
# 데이터 베이스 연결

conn = pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='root',  # 데이터베이스 비밀번호
    db='YES24',       # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

In [4]:
def sql_execute(sql, *args) :
    with conn.cursor() as cursor :
        cursor.execute(sql,*args)
        if 'select' in sql.lower() :
            return cursor.fetchall()
        conn.commit()

In [8]:
fail_list = []
sql = """
        INSERT INTO Books(title, author,publisher,publishing,rating,review, sales,price,ranking,ranking_weeks)
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
for page_num in range(1,43) :
    print('페이지 번호 :',page_num)
    browser.get(f"https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page_num}&pageSize=24") #URL로 이동
    #페이지가 로딩 될때까지 10초간 대기, 로딩이 완료되면 다음 코드 즉시 실행
    browser.implicitly_wait(time_to_wait=10) 
    book_list = browser.find_element(By.ID, 'yesBestList').find_elements(By.CLASS_NAME, 'itemUnit')
    for i in range(len(book_list)) :
        book_list[i].find_element(By.CLASS_NAME, 'gd_name').click() #제목 클릭하여 접속
        browser.implicitly_wait(time_to_wait=10) 
        try :
            title = browser.find_element(By.CLASS_NAME, 'gd_name').text # 제목
        except : 
            fail_list.append(i)
            continue
        try :
            author = browser.find_element(By.CLASS_NAME, 'gd_auth').text # 저자
        except : 
            fail_list.append(i)
            continue
        try :
            publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text #출판사
        except : 
            fail_list.append(i)
            continue
        try : 
            publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text #출판일
            match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
            if match :
                year, month, day = match.groups()
                date_data = datetime(int(year), int(month), int(day))
                publishing = date_data.strftime('%Y-%m-%d')
            else :
                date_data = datetime(int(9999), int(12), int(31))
                publishing = date_data.strftime('%Y-%m-%d')
        except : 
            fail_list.append(i)
            continue
        try :
            rating = browser.find_element(By.CLASS_NAME, 'gd_rating').find_element(By.CLASS_NAME,'yes_b').text # 평점
        except : 
            rating = '0.0'
        try :
            review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.CLASS_NAME, 'txC_blue').text.replace(',','')#리뷰수
        except : 
            review = '0'
        try :
            sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2].replace(',','') #판매지수
        except : 
            sales = '0'
        try :
            price = browser.find_element(By.CLASS_NAME, 'nor_price').find_element(By.CLASS_NAME, 'yes_m').text.replace(',','') #가격
        except : 
            fail_list.append(i)
            continue
        try :
            ranking = browser.find_element(By.CLASS_NAME, 'gd_best_tp02').text.split(' | ')[0]
            ranking = ''.join(filter(str.isdigit,ranking))#국내 도서랭킹
        except : 
            fail_list.append(i)
            continue
        try :
            ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best_tp02').text.split(' | ')[1] #국내 TOP 100
            ranking_weeks = ''.join(filter(str.isdigit,ranking_weeks))
        except : 
            ranking_weeks = '0'
        browser.back()
        browser.implicitly_wait(time_to_wait=10) 
        sql_execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        print('저장 정보 :',title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)

1
24
불변의 법칙 모건 하우절 저/이수경 역 서삼독 2024-02-28 9.8 133 493959 22500 1 15
나를 소모하지 않는 현명한 태도에 관하여 마티아스 뇔케 저/이미옥 역 퍼스트펭귄 2024-03-10 9.6 58 297954 16020 2 204
빨모쌤의 라이브 영어회화 신용하 저 웅진지식하우스 2024-04-24 9.7 25 105840 19800 3 0
나는 읽고 쓰고 버린다 손웅정 저 난다 2024-04-20 9.8 12 100740 15300 4 0
일류의 조건 사이토 다카시 저/정현 역 필름 2024-03-20 9.7 207 278133 17820 5 204
꽃길이 따로 있나, 내 삶이 꽃인 것을 오평선 저 포레스트북스 2024-03-22 9.8 55 147033 15120 6 202
사카모토 데이즈 15 더블특전판 스즈키 유우토 글그림 대원 2024-04-30 0.0 0 56190 5400 7 0
삶이 흔들릴 때 뇌과학을 읽습니다 이케가야 유지 글/김성기 역 힉스 2024-03-27 10.0 22 78048 16200 8 1001
마흔에 읽는 쇼펜하우어 강용수 저 유노북스 2023-09-07 8.8 432 855939 15300 9 112
세이노의 가르침 세이노(SayNo) 저 데이원 2023-03-02 9.0 2182 1649172 6480 10 122
나는 메트로폴리탄 미술관의 경비원입니다 패트릭 브링리 저/김희정, 조현주 역 웅진지식하우스 2023-11-24 9.2 281 421230 15750 11 2016
흔한남매 16 흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수 미래엔아이세움 2024-04-25 10.0 2 38040 13050 12 0
나는 나의 스무 살을 가장 존중한다 이하영 저 토네이도 2024-02-28 9.5 69 115827 16200 13 202
이상한 과자 가게 전천당 20 히로시마 레이코 글/쟈쟈 그림/김정화 역 길벗스쿨 2024-04-22 9.1 16 61239 11700 14 201
돈의 

: 

: 

In [6]:
title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks

('세이노의 가르침',
 '세이노(SayNo) 저',
 '데이원',
 '2023-03-02',
 '9.0',
 '2,182',
 '1649172',
 '6480',
 '10',
 '122')

In [7]:
review

'2,182'